# Sequential SAR Change Detection

In [ ]:
import ee
ee.Authenticate()

In [1]:
%matplotlib inline
from auxil.eeSar_seq import run
run()

Map(center=[51.0, 6.4], controls=(MeasureControl(options=['active_color', 'capture_z_index', 'completed_color'…

# How to use the interface

#### To generate change maps:

    - Select the desired time interval
    - Use the Polygon or rectangle tool to select an ROI (default: Jülich, Germany and surroundings). Multiple polygons are accepted. Navigate  to different parts of the world with the GoTo button
    - Leave the relative orbit value at 0
    - Choose orbit pass (ascending or descending), platform (A, B or Both)
    - Choose change map type for previewing (color-coded time of first change, last change or change frequency) or a bitemporal map for any of the consecutive time intervals
    - Set a desired opacity for the preview overlay
    - Leave the significance level at its default value
    - Select or deselect whether to show initially a minimum cloud cover S2 image. If deselected, a temporal average of all S1 images found is displayed
    - Press Collect to accumulate the images
Information about the images found is shown in the output window and an S2 image or a composite of the mean of all of the images is overlayed onto the map. If more than one relative orbit number is present, you can choose one of them by entering its value into the mask. For change detection, it is advisable not to use more than one orbit. 

    - Press Collect again if desired
    - Press Preview to see a preview overlay of the chosen change map 
Depending on the size of the images and their number, this can take up to a few minutes. The overlay is built up tile-by-tile as the GEE servers complete their tasks. Selecting a different change map or opacity does not require a new Run: the Preview button remains activated. You can use a water mask and/or make the no change pixels transparent with the corresponding check boxes. The color coding of the change maps is explained in the output window. When satisfied, enter username/path in the Export to assets mask or the filename in the export to drive mask. 

    - Press Review to examine an already exported asset
Use the same controls as for Preview to see the exported change maps in the asset folder. This is mach faster, as the calculations have already been performed at the nominal scale.    
    
    - Select ExportToAssets or ExportToDrive to start an export task on the GEE servers
An additional metadata file in CSV format is exported to the Google Drive for each asset/drive export. 

    - Select ExportATSF to export a temporal filter of the last image in the time series
The exported image contains the mean value of every pixel from the time of the most recent significant change. For comparison, the unfiltered image is also exported, along with a hybrid version which uses the Refined Lee filter for recently changed pixels and a log image which codes for each pixel the number of images uses in the time averaging.

    - Select PlotFromAsset
To generate change plots from any asset previously exported from the application. The graphs are discrimiated by Loewner order (positive definite, negative definite or indefinite) are saved in the home directory under the name of the asset itself. 

    - Select ClearPoly
To erase all existing polygons.    

    - Select EstimateENL
To calculate the ENL in the current polygon. The region chosen should be small (not larger than abou 50ha) to avoid timeouts.    

#### Other platforms

You can also analyse your own image collection, if available (e.g. fully dual pol sentinel-1 images downloaded from RSA and pre-processed wth the SNAP toolbox, or preprocessed RadarSat-2 imagery). Enter the collection name in the input template and the corresponding ENL. In order to enable the Collect button, draw an arbitrary polygon on the map.

#### File structure

The exported or downloaded, per-pixel change maps for a time sequence of length k are combined into a single file in GeoTiff format:

    band 1 cmap: Period in which most recent significant change occured (integer 1 ... k-1)
    band 2 smap: Period in which the first significant change occurred (integer 1 ... k-1)
    bans 3 fmap: Number of significant changes (integer 1 ... k-1)
    bands 4 ... k+3: Changes in each interval (1=positive definite, 2=negative definite, 3=indefinite)
    band k+4: Sentinel 2 background image (band B8) if available, otherwise averaged VV band of the s1 series. (This is only appended on export to assets)
    
#### Fine adjustments

    - Select a different significance level
The significance level applies to the tests performed in each change interval. They are statistically independent for no-change observations. 

    -Select 5x5 median filter
The p-values used in the tests are filtered before testing. This has the effect of supressing single-pixel changes (salt-and-pepper effect).

    -Select Quick Preview
Preview results are calculated at the scale corresponding to the current zoom level. This invalidates the ENL value so that, in general, fewer changes are detected. However calculation time is shortened, which is conveneient for rough data exploration. Unselect in order to force calcuation at the nominal scale of the data. 

    -Select a stride
Analyse only a subset of the available images. Stride = 2 for every second image, = 3 for every third, etc. Changing the stride requires that Collect be repeated.

#### Limits

The ROI should be completely enclosed within the S1 full scenes involved in the calculation. The larger the ROI and the longer the time series, the longer the preview and export calculations. By experience, time series longer than about 75 will cause a GEE server internal error (due to stack overflow caused by the depth of iteration).

#### References 

Knut Conradsen, Allan Aasbjerg Nielsen and Henning Skriver (2016). Determining the points of change in time series of polarimetric SAR data. IEEE Transactions on Geoscience and Remote Sensing 54(5), 3007-3024 http://www2.imm.dtu.dk/pubdb/views/publication_details.php?id=6825

Morton Canty, Allan Aasbjeg Nielsen, Henning Skriver and Knut Conradsen (2019). Statistical Analysis of Changes in Sentinel-1 Time Series on the Google Earth Engine, Remote Sensing 12(46), 1-16 https://www.mdpi.com/2072-4292/12/1/46

